# Arboles de decisión

## Definir una serie de reglas secuenciales que permitan hacer predicciones
### Método Cart: Pruebas Nominales, Ordinal, Numérico
### Algoritmo de Hunt

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Ejemplo 1

In [4]:
fraude = pd.read_csv("Fraude.csv")
fraude

,ID,Reembolso,EdoCivil,Ingresos,Fraude
0,1,Sí,Soltero,125000,No
1,2,No,Casado,100000,No
2,3,No,Soltero,70000,No
3,4,Sí,Casado,120000,No
4,5,No,Divorciado,95000,Sí
5,6,No,Casado,60000,No
6,7,Sí,Divorciado,220000,No
7,8,No,Soltero,85000,Sí
8,9,No,Casado,75000,No
9,10,No,Soltero,90000,Sí


In [5]:
feature_cols = ["Reembolso", "EdoCivil", "Ingresos"]
x = fraude[feature_cols].values
y = fraude.Fraude

In [6]:
# Etiquetado de variables cualitativas
from sklearn import preprocessing

Cod_Reem = preprocessing.LabelEncoder()
Cod_Reem.fit(["Sí", "No"])
x[:,0] = Cod_Reem.transform(x[:,0])

Cod_EdoC = preprocessing.LabelEncoder()
Cod_EdoC.fit(["Soltero", "Casado", "Divorciado"]) ## Soltero = 2, Casado = 0, Divorciado = 1
x[:,1] = Cod_EdoC.transform(x[:,1])

In [8]:
## Creación de grupos de entrenamiento y prueba
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=1)

## Crear objeto de clasificación del árbol de decisión
clf = DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)

# Predicción de la respuesta para el grupo de prueba
y_pred = clf.predict(x_test)


In [10]:
# Comparación de predicción con y real
print(y_test)
print(y_pred)

2    No
9    Sí
6    No
Name: Fraude, dtype: object
['Sí' 'Sí' 'No']


In [11]:
# Matriz de confusión
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

array([[1, 1],
       [0, 1]], dtype=int64)

In [12]:
# Estadísticas de desempeño
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          No       1.00      0.50      0.67         2
          Sí       0.50      1.00      0.67         1

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3



In [14]:
# Graficación de las reglas del árbol de decisión
## Bibliotecas necesarias
!pip install graphviz
!pip install pydotplus

from sklearn import tree
import graphviz
import pydotplus

from PIL import Image

In [17]:
# Creación de información para el árbol
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=feature_cols, class_names = ["No fraude", "Fraude"])

# Creación de la gráfica del arbol
graph = pydotplus.graph_from_dot_data(dot_data)

# Creación del arbol PDF
graph.write_pdf("fraudes.pdf")
# Creación del arbol PNG
graph.write_png("fraudes.png")

image = Image.open("fraudes.png")
image.show()

## Ejemplo 2

In [20]:
col_names = ["pregnant","glucose", "bp", "skin", "insulin", "bmi", "pedigree", "age", "label"]
diabetes = pd.read_csv("diabetes.csv", header = None, names = col_names)
diabetes = diabetes.iloc[1:, :]
diabetes.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1


In [21]:
## Selección de las características
feature_cols = ["pregnant","insulin", "bmi","age", "glucose", "bp", "pedigree"]
x = diabetes[feature_cols]
y = diabetes.label

In [28]:
## Selección de grupos de entrenamiento y prueba
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

## Crear objeto de clasificación del árbol de decisión
clfG = DecisionTreeClassifier(criterion = "gini") ## Criterio Gini/Entropia
clfG = clfG.fit(x_train, y_train)

# Predicción de la respuesta para el grupo de prueba
y_predG = clfG.predict(x_test)

# Matriz de confusión
cmG = confusion_matrix(y_test, y_predG)
print(cmG)
print("-------------------------------------------------------------")
# Estadísticas de desempeño
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predG))

[[79 20]
 [25 30]]
-------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.80      0.78        99
           1       0.60      0.55      0.57        55

    accuracy                           0.71       154
   macro avg       0.68      0.67      0.67       154
weighted avg       0.70      0.71      0.70       154



In [25]:
# Creación de información para el árbol
dot_dataG = tree.export_graphviz(clfG, out_file=None, feature_names=feature_cols, class_names = ["Normal", "Diabetes"])

# Creación de la gráfica del arbol
graphG = pydotplus.graph_from_dot_data(dot_dataG)

# Creación del arbol PDF
graphG.write_pdf("diabetes.pdf")
# Creación del arbol PNG
graphG.write_png("diabetes.png")

image = Image.open("diabetes.png")
image.show()

## Optimización del desempeño del árbol de decisión


In [29]:
## Crear objeto de clasificación del árbol de decisión
clfE = DecisionTreeClassifier(criterion = "entropy", max_depth = 3) ## Criterio Gini/Entropia
clfE = clfE.fit(x_train, y_train)

# Predicción de la respuesta para el grupo de prueba
y_predE = clfE.predict(x_test)

# Matriz de confusión
cmE = confusion_matrix(y_test, y_predE)
print(cmE)
print("-------------------------------------------------------------")
# Estadísticas de desempeño
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predE))

[[88 11]
 [20 35]]
-------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        99
           1       0.76      0.64      0.69        55

    accuracy                           0.80       154
   macro avg       0.79      0.76      0.77       154
weighted avg       0.80      0.80      0.79       154



In [30]:
# Creación de información para el árbol
dot_dataE = tree.export_graphviz(clfE, out_file=None, feature_names=feature_cols, class_names = ["Normal", "Diabetes"])

# Creación de la gráfica del arbol
graphE = pydotplus.graph_from_dot_data(dot_dataE)

# Creación del arbol PDF
graphE.write_pdf("diabetes2.pdf")
# Creación del arbol PNG
graphE.write_png("diabetes2.png")

image = Image.open("diabetes2.png")
image.show()

In [32]:
# Adición de colores al árbol de decisiones
from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

dot_dataE = StringIO()
export_graphviz(clfE, out_file = dot_dataE, filled = True, rounded = True, special_characters = True,
                feature_names = feature_cols, class_names = ["Normal", "Diabetes"])

graphEc = pydotplus.graph_from_dot_data(dot_dataE.getvalue())

In [34]:
# Creación del arbol PNG
from PIL import Image
graphEc.write_png("diabetes3.png")

image = Image.open("diabetes3.png")
image.show()